In [45]:
import pandas as pd
import numpy as np
import os
import glob

# Konfigurasi Tampilan agar kolom tidak terpotong
pd.set_option('display.max_columns', 100)

# Cek file apa saja yang ada di folder raw
files = glob.glob('../data/raw/*.csv')
print("File ditemukan:", files)

File ditemukan: ['../data/raw\\19860105-20210731_laptime.csv', '../data/raw\\19860105-20210731_odds.csv', '../data/raw\\19860105-20210731_race_result.csv', '../data/raw\\20020615-20210731_corner_passing_order.csv']


In [46]:
# Ambil file yang namanya mengandung 'race_result' atau file csv pertama jika tidak ada
target_file = [f for f in files if 'race_result' in f]
if target_file:
    file_path = target_file[0]
else:
    file_path = files[0] # Fallback ambil file pertama

print(f"Sedang membaca file: {file_path}")

try:
    # Coba baca dengan UTF-8 dulu (Standar Kaggle modern)
    df = pd.read_csv(file_path, encoding='utf-8')
    print("Sukses load dengan UTF-8!")
except:
    try:
        # Jika gagal, coba Shift-JIS (Standar Jepang lama)
        df = pd.read_csv(file_path, encoding='shift_jis')
        print("Sukses load dengan Shift-JIS!")
    except:
        # Terakhir coba cp932 (Varian Shift-JIS Windows)
        df = pd.read_csv(file_path, encoding='cp932')
        print("Sukses load dengan CP932!")

# Tampilkan 3 baris pertama
df.head(3)

Sedang membaca file: ../data/raw\19860105-20210731_race_result.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_11296\4184097905.py:12: DtypeWarning: Columns (8,11,15,16,17,18,19,20,21,22,23,24,25,26,27,34,36,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')


Sukses load dengan UTF-8!


,レース馬番ID,レースID,レース日付,開催回数,競馬場コード,競馬場名,開催日数,競争条件,レース記号/[抽],レース記号/(馬齢),レース記号/牝,レース記号/(父),レース記号/(別定),レース記号/(混),レース記号/(ハンデ),レース記号/(抽),レース記号/(市),レース記号/(定量),レース記号/牡,レース記号/関東配布馬,レース記号/(指),レース記号/関西配布馬,レース記号/九州産馬,レース記号/見習騎手,レース記号/せん,レース記号/(国際),レース記号/[指],レース記号/(特指),レース番号,重賞回次,レース名,リステッド・重賞競走,障害区分,芝・ダート区分,芝・ダート区分2,右左回り・直線区分,内・外・襷区分,距離(m),天候,馬場状態1,馬場状態2,発走時刻,着順,着順注記,枠番,馬番,馬名,性別,馬齢,斤量,騎手,タイム,着差,1コーナー,2コーナー,3コーナー,4コーナー,上り,単勝,人気,馬体重,場体重増減,東西・外国・地方区分,調教師,馬主,賞金(万円)
0,19860101010102,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,4歳以上300万下,NaN,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN,11:05,1.0,NaN,2,2,ワクセイ,牡,4,55.0,柏崎正次,1:34.3,NaN,NaN,7.0,3.0,1.0,38.6,2.1,1.0,468.0,0.0,東,宮沢今朝,アイ・ケイ・テイ・オーナーズ,290.0
1,19860101010103,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,4歳以上300万下,NaN,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN,11:05,2.0,NaN,3,3,マツタカラオー,牡,4,55.0,国兼正浩,1:36.0,大,NaN,4.0,3.0,4.0,40.3,7.0,4.0,430.0,4.0,東,斎藤籌敬,松井健一,120.0
2,19860101010105,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,4歳以上300万下,NaN,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN,11:05,3.0,NaN,5,5,カンキョウヘルス,牡,4,55.0,古小路重,1:36.2,1.1/4,NaN,1.0,2.0,3.0,40.6,59.1,6.0,460.0,-4.0,西,境直行,塚原金治,73.0


In [12]:
# 1. Cek Dimensi Data (Baris x Kolom)
print(f"\n[1] Dimensi Data: {df.shape[0]} baris, {df.shape[1]} kolom")


[1] Dimensi Data: 1626811 baris, 66 kolom


In [13]:
# 2. Cek Tipe Data & Penggunaan Memori
print("\n[2] Info Tipe Data:")
df.info()


[2] Info Tipe Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1626811 entries, 0 to 1626810
Data columns (total 66 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   レース馬番ID      1626811 non-null  int64  
 1   レースID        1626811 non-null  int64  
 2   レース日付        1626811 non-null  object 
 3   開催回数         1626811 non-null  int64  
 4   競馬場コード       1626811 non-null  int64  
 5   競馬場名         1626811 non-null  object 
 6   開催日数         1626811 non-null  int64  
 7   競争条件         1626811 non-null  object 
 8   レース記号/[抽]    12834 non-null    object 
 9   レース記号/(馬齢)   827790 non-null   object 
 10  レース記号/牝      208613 non-null   object 
 11  レース記号/(父)    23423 non-null    object 
 12  レース記号/(別定)   361406 non-null   object 
 13  レース記号/(混)    753618 non-null   object 
 14  レース記号/(ハンデ)  95331 non-null    object 
 15  レース記号/(抽)    6016 non-null     object 
 16  レース記号/(市)    6180 non-null     object 
 17  レース記号/(定量)   324044 non-n

In [14]:
# 3. Cek Missing Values (Data Kosong)
print("\n[3] Jumlah Missing Values per Kolom:")
missing_vals = df.isnull().sum()
print(missing_vals[missing_vals > 0])


[3] Jumlah Missing Values per Kolom:
レース記号/[抽]      1613977
レース記号/(馬齢)      799021
レース記号/牝        1418198
レース記号/(父)      1603388
レース記号/(別定)     1265405
レース記号/(混)       873193
レース記号/(ハンデ)    1531480
レース記号/(抽)      1620795
レース記号/(市)      1620631
レース記号/(定量)     1302767
レース記号/牡        1624410
レース記号/関東配布馬    1626023
レース記号/(指)      1596403
レース記号/関西配布馬    1625624
レース記号/九州産馬     1624349
レース記号/見習騎手     1619273
レース記号/せん       1626787
レース記号/(国際)     1584437
レース記号/[指]       758933
レース記号/(特指)     1479642
重賞回次           1562869
リステッド・重賞競走     1560707
障害区分           1572791
芝・ダート区分2       1593502
右左回り・直線区分        52336
内・外・襷区分        1451916
馬場状態2          1593502
着順               14915
着順注記           1611334
タイム              14584
着差              136760
1コーナー           929579
2コーナー           837495
3コーナー            18769
4コーナー             9376
上り               25754
単勝                6438
人気                6438
馬体重               4250
場体重増減             4250
賞金(万円)         1016845
dtype: int64


In [15]:
# 4. Cek Duplikasi Data
duplicate_count = df.duplicated().sum()
print(f"\n[4] Jumlah Baris Duplikat: {duplicate_count}")


[4] Jumlah Baris Duplikat: 0


In [16]:
# 5. Cek Integritas Kolom Kunci (Primary Keys)
if 'レースID' in df.columns and '馬番' in df.columns:
    unique_check = df.duplicated(subset=['レースID', '馬番']).sum()
    print(f"\n[5] Cek Konsistensi ID (Race ID + Horse No):")
    if unique_check == 0:
        print("    OK! Tidak ada duplikasi pada kombinasi Race ID dan Nomor Kuda.")
    else:
        print(f"    WARNING! Ada {unique_check} data ganda untuk kuda yang sama di race yang sama.")


[5] Cek Konsistensi ID (Race ID + Horse No):
    OK! Tidak ada duplikasi pada kombinasi Race ID dan Nomor Kuda.


In [17]:
# 6. Cek Nilai Unik Kolom Kategorikal
target_cols_check = ['芝・ダート区分', '天候', '馬場状態1', '性別']
print("\n[6] Cek Nilai Unik untuk Mapping:")

for col in target_cols_check:
    if col in df.columns:
        print(f"   > Kolom '{col}': {df[col].unique()}")
    else:
        print(f"   > Kolom '{col}' tidak ditemukan.")


[6] Cek Nilai Unik untuk Mapping:
   > Kolom '芝・ダート区分': ['ダート' '芝']
   > Kolom '天候': ['晴' '曇' '雨' '小雨' '小雪' '雪' '晴 ' '曇 ' '小雨 ']
   > Kolom '馬場状態1': ['良' '稍重' '不良' '重']
   > Kolom '性別': ['牡' '牝' 'セ']


In [ ]:
import pandas as pd

print("=== TECHNICAL EDA REPORT ===")

# 1. Cek Dimensi Data (Baris x Kolom)
print(f"\n[1] Dimensi Data: {df.shape[0]} baris, {df.shape[1]} kolom")

# 2. Cek Tipe Data & Penggunaan Memori
print("\n[2] Info Tipe Data:")
df.info()

# 3. Cek Missing Values (Data Kosong)
print("\n[3] Jumlah Missing Values per Kolom:")
missing_vals = df.isnull().sum()
print(missing_vals[missing_vals > 0])

# 4. Cek Duplikasi Data
duplicate_count = df.duplicated().sum()
print(f"\n[4] Jumlah Baris Duplikat: {duplicate_count}")

# 5. Cek Integritas Kolom Kunci (Primary Keys)
if 'レースID' in df.columns and '馬番' in df.columns:
    unique_check = df.duplicated(subset=['レースID', '馬番']).sum()
    print(f"\n[5] Cek Konsistensi ID (Race ID + Horse No):")
    if unique_check == 0:
        print("    OK! Tidak ada duplikasi pada kombinasi Race ID dan Nomor Kuda.")
    else:
        print(f"    WARNING! Ada {unique_check} data ganda untuk kuda yang sama di race yang sama.")

# 6. Cek Nilai Unik Kolom Kategorikal
target_cols_check = ['芝・ダート区分', '天候', '馬場状態1', '性別']
print("\n[6] Cek Nilai Unik untuk Mapping:")

for col in target_cols_check:
    if col in df.columns:
        print(f"   > Kolom '{col}': {df[col].unique()}")
    else:
        print(f"   > Kolom '{col}' tidak ditemukan.")

print("\n=== END OF REPORT ===")

=== TECHNICAL EDA REPORT ===

[1] Dimensi Data: 1626811 baris, 66 kolom

[2] Info Tipe Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1626811 entries, 0 to 1626810
Data columns (total 66 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   レース馬番ID      1626811 non-null  int64  
 1   レースID        1626811 non-null  int64  
 2   レース日付        1626811 non-null  object 
 3   開催回数         1626811 non-null  int64  
 4   競馬場コード       1626811 non-null  int64  
 5   競馬場名         1626811 non-null  object 
 6   開催日数         1626811 non-null  int64  
 7   競争条件         1626811 non-null  object 
 8   レース記号/[抽]    12834 non-null    object 
 9   レース記号/(馬齢)   827790 non-null   object 
 10  レース記号/牝      208613 non-null   object 
 11  レース記号/(父)    23423 non-null    object 
 12  レース記号/(別定)   361406 non-null   object 
 13  レース記号/(混)    753618 non-null   object 
 14  レース記号/(ハンデ)  95331 non-null    object 
 15  レース記号/(抽)    6016 non-null     object 
 1

In [47]:
import pandas as pd
import numpy as np

# 1. Dictionary Mapping (KHUSUS Dataset Anda)
# Kiri: Header Jepang (dari tabel Anda) -> Kanan: Header Inggris Baru
column_mapping = {
    # --- Identitas Balapan (Race Metadata) ---
    'レースID': 'race_id',              # ID Unik
    'レース日付': 'date',                 # Tanggal (Penting untuk Time Series split)
    '発走時刻': 'start_time',           # Jam Lari (Pagi/Sore pengaruh ke suhu/track)
    '競馬場名': 'venue',                # Lokasi (Sapporo, Tokyo, dll)
    '開催回数': 'holding_number',       # Kai (Event ke-berapa musim ini)
    '開催日数': 'holding_days',         # Nichi (Hari ke-berapa dalam event)
    'レース名': 'race_name',            # Nama Lomba
    '競争条件': 'race_class',           # Kelas (Maiden, 1 Win Class, Open, dll) - PENTING
    'リステッド・重賞競走': 'race_type', # Tipe (Listed/Graded/Null)
    '重賞回次': 'grade_race_count',     # Hitungan ke-berapa untuk G1/G2
    
    # --- Kondisi Lintasan (Track Info) ---
    '芝・ダート区分': 'surface',          # Turf/Dirt
    '距離(m)': 'distance',              # Jarak
    '右左回り・直線区分': 'course_direction', # Kanan/Kiri/Lurus (Penting: kuda punya preferensi)
    '内・外・襷区分': 'course_track',     # Jalur Dalam/Luar (Penting untuk strategi)
    '天候': 'weather',                  # Cuaca
    '馬場状態1': 'track_condition',     # Kondisi Trek (Good/Heavy)
    '馬場状態2': 'track_condition_detail', # Detail (jarang ada, tapi ambil saja)

    # --- Profil Peserta (Horse/Jockey Profile) ---
    '馬名': 'horse_name',               # Nama Kuda
    '性別': 'sex',                      # Gender
    '馬齢': 'age',                      # Umur
    '東西・外国・地方区分': 'region',     # Asal (Timur/Barat/Asing) - PENTING (Kuda Kansai vs Kanto)
    '調教師': 'trainer',                # Pelatih
    '馬主': 'owner',                    # Pemilik (Sumber daya/uang pemilik pengaruh ke performa)
    '騎手': 'jockey',                   # Joki
    '斤量': 'jockey_weight',            # Beban bawaan
    '枠番': 'frame_no',                 # Nomor Frame (Posisi start pengaruh besar)
    '馬番': 'horse_no',                 # Nomor Punggung

    # --- Hasil & Taktik (Result & Tactics) ---
    '着順': 'rank',                     # Target Prediksi
    '着順注記': 'rank_note',            # Catatan (Diskualifikasi/Batal)
    'タイム': 'time',                   # Waktu Finish
    '着差': 'margin',                   # Jarak Kemenangan
    '上り': 'last_3f',                  # Kecepatan 600m terakhir (Stamina)
    
    # --- Taktik (Posisi Tikungan) ---
    # Ini fitur "Emas" untuk menentukan gaya lari (Nige/Senkou/Sashi/Oikomi)
    '1コーナー': 'corner_1',
    '2コーナー': 'corner_2',
    '3コーナー': 'corner_3',
    '4コーナー': 'corner_4',

    # --- Indikator Pasar (Betting) ---
    '単勝': 'odds',                     # Win Odds
    '人気': 'popularity',               # Favorit Pasar

    # --- Kondisi Fisik Harian ---
    '馬体重': 'horse_weight',           # Berat Badan
    '場体重増減': 'weight_change',        # Naik/Turun Berat (Kesehatan)
    '賞金(万円)': 'prize'                 # Hadiah (Indikator kelas kuda)
}

# 2. Rename Kolom dari Bahasa Jepang ke Inggris
df_clean = df.rename(columns=column_mapping)

# 3. FILTER PINTAR: Ambil hanya kolom yang ada di daftar mapping
target_cols = list(column_mapping.values())
df_clean = df_clean[target_cols]

# 4. Cek Hasil
print(f"Dimensi Data Bersih: {df_clean.shape}")
print("Header kolom sekarang:")
print(df_clean.columns.tolist())

# Tampilkan 3 baris teratas untuk verifikasi
df_clean.head(3)

Dimensi Data Bersih: (1626811, 41)
Header kolom sekarang:
['race_id', 'date', 'start_time', 'venue', 'holding_number', 'holding_days', 'race_name', 'race_class', 'race_type', 'grade_race_count', 'surface', 'distance', 'course_direction', 'course_track', 'weather', 'track_condition', 'track_condition_detail', 'horse_name', 'sex', 'age', 'region', 'trainer', 'owner', 'jockey', 'jockey_weight', 'frame_no', 'horse_no', 'rank', 'rank_note', 'time', 'margin', 'last_3f', 'corner_1', 'corner_2', 'corner_3', 'corner_4', 'odds', 'popularity', 'horse_weight', 'weight_change', 'prize']


,race_id,date,start_time,venue,holding_number,holding_days,race_name,race_class,race_type,grade_race_count,surface,distance,course_direction,course_track,weather,track_condition,track_condition_detail,horse_name,sex,age,region,trainer,owner,jockey,jockey_weight,frame_no,horse_no,rank,rank_note,time,margin,last_3f,corner_1,corner_2,corner_3,corner_4,odds,popularity,horse_weight,weight_change,prize
0,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,ダート,1500,右,NaN,晴,良,NaN,ワクセイ,牡,4,東,宮沢今朝,アイ・ケイ・テイ・オーナーズ,柏崎正次,55.0,2,2,1.0,NaN,1:34.3,NaN,38.6,NaN,7.0,3.0,1.0,2.1,1.0,468.0,0.0,290.0
1,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,ダート,1500,右,NaN,晴,良,NaN,マツタカラオー,牡,4,東,斎藤籌敬,松井健一,国兼正浩,55.0,3,3,2.0,NaN,1:36.0,大,40.3,NaN,4.0,3.0,4.0,7.0,4.0,430.0,4.0,120.0
2,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,ダート,1500,右,NaN,晴,良,NaN,カンキョウヘルス,牡,4,西,境直行,塚原金治,古小路重,55.0,5,5,3.0,NaN,1:36.2,1.1/4,40.6,NaN,1.0,2.0,3.0,59.1,6.0,460.0,-4.0,73.0


In [48]:
# --- BLOCK: VALIDASI DAN MAPPING KATEGORIKAL ---

# Daftar kolom kategorikal yang akan kita proses
target_cat_cols = ['sex', 'weather', 'surface', 'track_condition', 'course_direction', 'region']

# ---------------------------------------------------------
# 1. CEK UNIQUE VALUES SEBELUM MAPPING (Data Asli)
# ---------------------------------------------------------

# Menghapus spasi kosong di awal/akhir teks
obj_cols = df_clean.select_dtypes(include=['object']).columns
for col in obj_cols:
    df_clean[col] = df_clean[col].str.strip()

print("=== [1] NILAI UNIK SEBELUM MAPPING (BAHASA JEPANG) ===")
for col in target_cat_cols:
    if col in df_clean.columns:
        unique_vals = df_clean[col].unique()
        print(f" > Kolom '{col}' ({len(unique_vals)} variasi): {unique_vals}")
    else:
        print(f" > Kolom '{col}' tidak ditemukan di DataFrame.")

=== [1] NILAI UNIK SEBELUM MAPPING (BAHASA JEPANG) ===
 > Kolom 'sex' (3 variasi): ['牡' '牝' 'セ']
 > Kolom 'weather' (6 variasi): ['晴' '曇' '雨' '小雨' '小雪' '雪']
 > Kolom 'surface' (2 variasi): ['ダート' '芝']
 > Kolom 'track_condition' (4 variasi): ['良' '稍重' '不良' '重']
 > Kolom 'course_direction' (4 variasi): ['右' nan '左' '直線']
 > Kolom 'region' (4 variasi): ['東' '西' '外' '地']


In [49]:
# 2. DEFINISI KAMUS (DICTIONARY)
sex_map = {
    '牡': 'Male',
    '牝': 'Female',
    'セ': 'Gelding'
}

weather_map = {
    '晴': 'Sunny',
    '曇': 'Cloudy',
    '雨': 'Rain',
    '小雨': 'Light Rain',
    '雪': 'Snow',
    '小雪': 'Light Snow'
}

surface_map = {
    '芝': 'Turf',
    'ダート': 'Dirt',
    '障害': 'Steeplechase'
}

track_condition_map = {
    '良': 'Firm',      
    '稍重': 'Good',
    '重': 'Soft',
    '不良': 'Heavy'    
}

course_direction_map = {
    '右': 'Right',
    '左': 'Left',
    '直線': 'Straight'
}

region_map = {
    '東': 'East (Miho)',
    '西': 'West (Ritto)',
    '地': 'Local',
    '外': 'Foreign'
}

mappings = {
    'sex': sex_map,
    'weather': weather_map,
    'surface': surface_map,
    'track_condition': track_condition_map,
    'course_direction': course_direction_map,
    'region': region_map
}

In [50]:
# 3. EKSEKUSI MAPPING
for col, map_dict in mappings.items():
    if col in df_clean.columns:
        # Lakukan mapping
        # fillna(df_clean[col]) digunakan agar jika ada data yang TIDAK ada di kamus, 
        # dia tidak jadi NaN, tapi tetap huruf Jepang aslinya (agar bisa kita deteksi errornya)
        df_clean[col] = df_clean[col].map(map_dict).fillna(df_clean[col])

print(" > Mapping selesai.")

# 4. VERIFIKASI HASIL (CEK ULANG)
print("\n=== [HASIL AKHIR] NILAI UNIK SETELAH MAPPING ===")
target_cat_cols = ['sex', 'weather', 'surface', 'track_condition', 'course_direction', 'region']

for col in target_cat_cols:
    if col in df_clean.columns:
        vals = df_clean[col].unique()
        print(f"[{col}] -> {vals}")

 > Mapping selesai.

=== [HASIL AKHIR] NILAI UNIK SETELAH MAPPING ===
[sex] -> ['Male' 'Female' 'Gelding']
[weather] -> ['Sunny' 'Cloudy' 'Rain' 'Light Rain' 'Light Snow' 'Snow']
[surface] -> ['Dirt' 'Turf']
[track_condition] -> ['Firm' 'Good' 'Heavy' 'Soft']
[course_direction] -> ['Right' nan 'Left' 'Straight']
[region] -> ['East (Miho)' 'West (Ritto)' 'Foreign' 'Local']


In [51]:
df_clean.tail(20)

,race_id,date,start_time,venue,holding_number,holding_days,race_name,race_class,race_type,grade_race_count,surface,distance,course_direction,course_track,weather,track_condition,track_condition_detail,horse_name,sex,age,region,trainer,owner,jockey,jockey_weight,frame_no,horse_no,rank,rank_note,time,margin,last_3f,corner_1,corner_2,corner_3,corner_4,odds,popularity,horse_weight,weight_change,prize
1626791,202110030611,2021-07-18,15:35:00,小倉,3,6,トヨタ賞中京記念,3歳以上オープン,G3,69.0,Turf,1800,Right,NaN,Cloudy,Firm,NaN,アメリカズカップ,Male,7,West (Ritto),音無秀孝,谷掛龍夫,松若風馬,54.0,6,7,9.0,NaN,1:47.1,3/4,35.3,2.0,3.0,2.0,4.0,117.4,12.0,452.0,-2.0,NaN
1626792,202110030611,2021-07-18,15:35:00,小倉,3,6,トヨタ賞中京記念,3歳以上オープン,G3,69.0,Turf,1800,Right,NaN,Cloudy,Firm,NaN,ダノンチェイサー,Male,5,West (Ritto),池江泰寿,ダノックス,岩田望来,56.0,2,2,10.0,NaN,1:47.2,3/4,35.1,5.0,5.0,6.0,4.0,14.0,7.0,496.0,8.0,NaN
1626793,202110030611,2021-07-18,15:35:00,小倉,3,6,トヨタ賞中京記念,3歳以上オープン,G3,69.0,Turf,1800,Right,NaN,Cloudy,Firm,NaN,アバルラータ,Female,7,West (Ritto),鮫島一歩,安原浩司,西村淳也,52.0,8,12,11.0,NaN,1:47.4,1,34.6,11.0,11.0,10.0,11.0,89.2,11.0,462.0,4.0,NaN
1626794,202110030611,2021-07-18,15:35:00,小倉,3,6,トヨタ賞中京記念,3歳以上オープン,G3,69.0,Turf,1800,Right,NaN,Cloudy,Firm,NaN,ドリームソルジャー,Male,7,West (Ritto),渡辺薫彦,カナヤマホールディングス,鮫島克駿,54.0,4,4,12.0,NaN,1:47.6,1.1/4,34.8,11.0,11.0,10.0,11.0,78.8,10.0,480.0,0.0,NaN
1626795,202110030612,2021-07-18,16:15:00,小倉,3,6,3歳以上1勝クラス,3歳以上1勝クラス,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,キョウワウォール,Male,5,West (Ritto),飯田雄三,浅川昌彦,中井裕二,57.0,7,13,1.0,NaN,1:46.2,NaN,38.4,13.0,12.0,6.0,6.0,18.5,8.0,476.0,-12.0,760.0
1626796,202110030612,2021-07-18,16:15:00,小倉,3,6,3歳以上1勝クラス,3歳以上1勝クラス,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,パイプライナー,Male,4,West (Ritto),杉山晴紀,ＣＨＥＶＡＬＡＴＴＡＣＨＥ,松若風馬,57.0,6,11,2.0,NaN,1:46.5,2,38.5,6.0,6.0,9.0,7.0,9.0,4.0,508.0,2.0,300.0
1626797,202110030612,2021-07-18,16:15:00,小倉,3,6,3歳以上1勝クラス,3歳以上1勝クラス,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,サワヤカコーズサン,Male,3,West (Ritto),茶木太樹,永井啓弍,団野大成,54.0,2,4,3.0,NaN,1:46.6,3/4,39.3,2.0,2.0,2.0,2.0,3.9,2.0,466.0,6.0,190.0
1626798,202110030612,2021-07-18,16:15:00,小倉,3,6,3歳以上1勝クラス,3歳以上1勝クラス,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,タイキフォース,Male,3,West (Ritto),牧浦充徳,大樹ファーム,浜中俊,54.0,1,2,4.0,NaN,1:46.9,1.1/2,39.3,5.0,4.0,5.0,3.0,3.4,1.0,478.0,8.0,110.0
1626799,202110030612,2021-07-18,16:15:00,小倉,3,6,3歳以上1勝クラス,3歳以上1勝クラス,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,メイショウメイユウ,Male,4,West (Ritto),荒川義之,松本好雄,川島信二,57.0,4,8,5.0,NaN,1:47.2,2,39.8,16.0,15.0,2.0,3.0,110.3,15.0,446.0,0.0,76.0
1626800,202110030612,2021-07-18,16:15:00,小倉,3,6,3歳以上1勝クラス,3歳以上1勝クラス,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,ソジュン,Male,3,West (Ritto),安田翔伍,山口功一郎,藤岡康太,54.0,4,7,6.0,NaN,1:47.4,1.1/4,40.3,10.0,9.0,1.0,1.0,68.4,13.0,518.0,1.0,NaN


In [55]:
# 6. Cek Nilai Unik Semua Kolom di df_clean
print(f"\n[6] Cek Nilai Unik untuk Semua Kolom ({len(df_clean.columns)} kolom):")

for col in df_clean.columns:
    unique_vals = df_clean[col].unique()
    num_unique = len(unique_vals)
    
    # Menampilkan info: Nama Kolom (Jumlah Unik) -> Isinya
    # Limit isi print agar tidak terlalu panjang jika datanya ribuan
    if num_unique > 30:
        print(f"   > Kolom '{col}': {num_unique} nilai unik (Terlalu banyak untuk ditampilkan)")
    else:
        print(f"   > Kolom '{col}': {unique_vals}")



[6] Cek Nilai Unik untuk Semua Kolom (41 kolom):
   > Kolom 'race_id': 121938 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'date': 3786 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'start_time': 208 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'venue': ['札幌' '函館' '福島' '新潟' '東京' '中山' '中京' '京都' '阪神' '小倉']
   > Kolom 'holding_number': [1 2 3 4 5 6 7 8]
   > Kolom 'holding_days': [ 1  2  3  4  5  6  7  8  9 10 11 12]
   > Kolom 'race_name': 2330 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'race_class': 49 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'race_type': [nan 'G3' 'G' 'G2' 'G1' 'J.G3' 'J.G2' 'J.G1' 'L']
   > Kolom 'grade_race_count': 164 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'surface': ['Dirt' 'Turf']
   > Kolom 'distance': 66 nilai unik (Terlalu banyak untuk ditampilkan)
   > Kolom 'course_direction': ['Right' nan 'Left' 'Straight']
   > Kolom 'course_track': [nan '外' '外-内' '襷' '内2周' '内-外' '外2周']
   >

In [53]:
df_clean.nunique()

race_id                   121938
date                        3786
start_time                   208
venue                         10
holding_number                 8
holding_days                  12
race_name                   2330
race_class                    49
race_type                      8
grade_race_count             163
surface                        2
distance                      66
course_direction               3
course_track                   6
weather                        6
track_condition                4
track_condition_detail         4
horse_name                147495
sex                            3
age                           14
region                         4
trainer                     1332
owner                       6579
jockey                      1249
jockey_weight                 41
frame_no                       8
horse_no                      24
rank                          24
rank_note                      6
time                        2401
margin    

In [56]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm # Import tqdm

# 1. Inisialisasi tqdm untuk Pandas
tqdm.pandas()

# --- B. Konversi Waktu (1:34.3 -> 94.3 detik) ---
def clean_time(t_str):
    if pd.isna(t_str): return None
    t_str = str(t_str)
    try:
        if ':' in t_str:
            parts = t_str.split(':')
            return float(parts[0]) * 60 + float(parts[1])
        return float(t_str)
    except:
        return None

print("2. Konversi waktu...")
# Kita pakai progress_apply disini juga biar terlihat prosesnya
df_clean['time_seconds'] = df_clean['time'].progress_apply(clean_time)


# --- C. Bersihkan Rank (Posisi) ---
df_clean['rank'] = pd.to_numeric(df_clean['rank'], errors='coerce')


# --- D. Translasi Nama Kuda (BAGIAN TERBERAT) ---
try:
    import pykakasi
    kks = pykakasi.kakasi()
    
    def jp_to_romaji(text):
        if not isinstance(text, str): return text
        result = kks.convert(text)
        return " ".join([item['hepburn'] for item in result]).title()

    print("\n3. Menerjemahkan Nama Kuda & Joki (Mohon tunggu)...")
    
    # --- PERUBAHAN UTAMA DISINI: Menggunakan .progress_apply() ---
    print("   > Translating Horse Names:")
    df_clean['horse_name_eng'] = df_clean['horse_name'].progress_apply(jp_to_romaji)
    
    print("   > Translating Jockey Names:")
    df_clean['jockey_eng'] = df_clean['jockey'].progress_apply(jp_to_romaji)

    print("   > Translating Jockey Names:")
    df_clean['race_name_eng'] = df_clean['race_name'].progress_apply(jp_to_romaji)

    print("   > Translating Jockey Names:")
    df_clean['race_class_eng'] = df_clean['race_class'].progress_apply(jp_to_romaji)

    print("   > Translating Jockey Names:")
    df_clean['owner_eng'] = df_clean['owner'].progress_apply(jp_to_romaji)
    
except ImportError:
    print("Library pykakasi belum terinstall, nama tetap Kanji.")

# --- HASIL AKHIR ---
print("\n--- Data Siap ---")
# Drop baris yang gagal finish (rank NaN) atau waktu error
df_final = df_clean.dropna(subset=['rank', 'time_seconds']) 

print(f"Total Baris Bersih: {len(df_final)}")
display(df_final[['date', 'venue', 'horse_name_eng', 'sex', 'rank', 'time_seconds']].head())

d:\KULIAH\FINAL PROJECT\Project-UMA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Konversi waktu...


100%|██████████| 1626811/1626811 [00:04<00:00, 350051.51it/s]



3. Menerjemahkan Nama Kuda & Joki (Mohon tunggu)...
   > Translating Horse Names:


100%|██████████| 1626811/1626811 [05:50<00:00, 4637.04it/s]


...

100%|██████████| 1626811/1626811 [00:52<00:00, 30761.69it/s]


   > Translating Jockey Names:


100%|██████████| 1626811/1626811 [06:34<00:00, 4127.84it/s]



--- Data Siap ---
Total Baris Bersih: 1611896


,date,venue,horse_name_eng,sex,rank,time_seconds
0,1986-06-07,札幌,Wakusei,Male,1.0,94.3
1,1986-06-07,札幌,Matsutakaraoo,Male,2.0,96.0
2,1986-06-07,札幌,Kankyouherusu,Male,3.0,96.2
3,1986-06-07,札幌,Suzutakaeesu,Male,4.0,96.6
4,1986-06-07,札幌,Kuriyaapaaman,Male,5.0,97.4


In [57]:
df_final.head(10)

,race_id,date,start_time,venue,holding_number,holding_days,race_name,race_class,race_type,grade_race_count,surface,distance,course_direction,course_track,weather,track_condition,track_condition_detail,horse_name,sex,age,region,trainer,owner,jockey,jockey_weight,frame_no,horse_no,rank,rank_note,time,margin,last_3f,corner_1,corner_2,corner_3,corner_4,odds,popularity,horse_weight,weight_change,prize,time_seconds,horse_name_eng,jockey_eng,race_name_eng,race_class_eng,owner_eng
0,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,ワクセイ,Male,4,East (Miho),宮沢今朝,アイ・ケイ・テイ・オーナーズ,柏崎正次,55.0,2,2,1.0,NaN,1:34.3,NaN,38.6,NaN,7.0,3.0,1.0,2.1,1.0,468.0,0.0,290.0,94.3,Wakusei,Kashiwazaki Masaji,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Ai・Kei・Tei・Oonaazu
1,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,マツタカラオー,Male,4,East (Miho),斎藤籌敬,松井健一,国兼正浩,55.0,3,3,2.0,NaN,1:36.0,大,40.3,NaN,4.0,3.0,4.0,7.0,4.0,430.0,4.0,120.0,96.0,Matsutakaraoo,Kuni Ken Masahiro,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Matsui Ken'Ichi
2,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,カンキョウヘルス,Male,4,West (Ritto),境直行,塚原金治,古小路重,55.0,5,5,3.0,NaN,1:36.2,1.1/4,40.6,NaN,1.0,2.0,3.0,59.1,6.0,460.0,-4.0,73.0,96.2,Kankyouherusu,Ko Kouji Juu,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Tsukahara Kinji
3,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,スズタカエース,Male,5,West (Ritto),坂口正大,鈴木隆,小谷内秀,57.0,6,6,4.0,NaN,1:36.6,2.1/2,41.2,NaN,2.0,1.0,2.0,2.1,2.0,456.0,14.0,44.0,96.6,Suzutakaeesu,Koyachi Shuu,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Suzuki Takashi
4,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,クリヤーパーマン,Male,4,West (Ritto),荻野光男,勢戸澄雄,猿橋重利,55.0,7,7,5.0,NaN,1:37.4,5,41.7,NaN,3.0,3.0,4.0,6.2,3.0,432.0,-6.0,29.0,97.4,Kuriyaapaaman,Sarubashi Shigetoshi,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Sei Ko Sumio
5,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,ラガーヒム,Male,4,West (Ritto),大久保正,奥村啓二,大久保光,55.0,1,1,6.0,NaN,1:38.1,4,41.5,NaN,6.0,6.0,6.0,59.1,6.0,436.0,6.0,NaN,98.1,Ragaahimu,Ookubo Hikari,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Okumura Keiji
6,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,レミーファイヤー,Male,4,West (Ritto),大根田裕,石田嘉平,松田幸春,55.0,4,4,7.0,NaN,1:39.2,7,42.7,NaN,4.0,6.0,7.0,24.2,5.0,450.0,4.0,NaN,99.2,Remiifaiyaa,Matsuda Yukiharu,4 Toshi Ijou 300 Man Shita,4 Toshi Ijou 300 Man Shita,Ishida Kahei
7,198601010102,1986-06-07,11:35,札幌,1,1,3歳新馬,3歳新馬,NaN,NaN,Dirt,1000,Right,NaN,Sunny,Firm,NaN,パンパードスキー,Female,3,East (Miho),藤原敏文,近嶋勇,津曲浩二,53.0,3,3,1.0,NaN,1:01.1,NaN,37.7,NaN,NaN,1.0,1.0,4.2,2.0,488.0,0.0,480.0,61.1,Panpaadosukii,Tsumagari Kouji,3 Toshi Shinba,3 Toshi Shinba,Kin Shima Isami
8,198601010102,1986-06-07,11:35,札幌,1,1,3歳新馬,3歳新馬,NaN,NaN,Dirt,1000,Right,NaN,Sunny,Firm,NaN,サクラハゴロモ,Female,3,East (Miho),境勝太郎,さくらコマース,小島太,53.0,5,5,2.0,NaN,1:01.5,2.1/2,37.6,NaN,NaN,3.0,3.0,1.2,1.0,444.0,0.0,190.0,61.5,Sakurahagoromo,Kojima Ta,3 Toshi Shinba,3 Toshi Shinba,Sakura Komaasu
9,198601010102,1986-06-07,11:35,札幌,1,1,3歳新馬,3歳新馬,NaN,NaN,Dirt,1000,Right,NaN,Sunny,Firm,NaN,オカノアスティー,Female,3,West (Ritto),山本正司,岡崎貞夫,松永幹夫,50.0,6,6,3.0,NaN,1:01.6,1/2,38.1,NaN,NaN,2.0,2.0,41.6,6.0,418.0,0.0,120.0,61.6,Okanoasuteii,Matsunaga Mikio,3 Toshi Shinba,3 Toshi Shinba,Okazaki Sadao


In [11]:
# Cek 10 nilai unik pertama di kolom sex (sebelum mapping jika memungkinkan, atau cek ulang dr awal)
# Kita reload sedikit bagian kolom sex dari df asli
print("Isi Unik Kolom Sex:", df['性別'].unique())

Isi Unik Kolom Sex: ['牡' '牝' 'セ']


In [64]:
# Keyword pencarian (Case insensitive)
keyword = "キタサンブラック"

# Filter DataFrame
# na=False agar tidak error jika ada nama yang kosong
search_results = df_clean[df_clean['horse_name'].str.contains(keyword, case=False, na=False)]

# Urutkan berdasarkan tanggal balapan
search_results = search_results.sort_values('date')

# Tampilkan kolom-kolom penting
cols_to_show = ['date', 'race_name_eng','race_name_eng','race_class_eng','owner_eng', 'horse_name_eng', 'jockey_eng', 'rank', 'odds', 'popularity']

print(f"Ditemukan {len(search_results)} data balapan dengan nama '{keyword}':")
display(search_results[cols_to_show])

Ditemukan 20 data balapan dengan nama 'キタサンブラック':


,date,race_name_eng,race_name_eng,race_class_eng,owner_eng,horse_name_eng,jockey_eng,rank,odds,popularity
1316227,2015-01-31,3 Toshi Shinba,3 Toshi Shinba,3 Toshi Shinba,Oono Shouji,Kitasanburakku,Gotou Hiroki,1.0,7.9,3.0
1317558,2015-02-22,3 Toshi 500 Man Shita,3 Toshi 500 Man Shita,3 Toshi 500 Man Shita,Oono Shouji,Kitasanburakku,Kitamura Hiroshi,1.0,48.4,9.0
1327498,2015-03-22,Fuji Tv Supuringu S,Fuji Tv Supuringu S,3 Toshi Oopun,Oono Shouji,Kitasanburakku,Kitamura Hiroshi,1.0,12.3,5.0
1328915,2015-04-19,Satsukishou,Satsukishou,3 Toshi Oopun,Oono Shouji,Kitasanburakku,Hamanaka Shun,3.0,9.7,4.0
1319806,2015-05-31,Toukyouyuushun,Toukyouyuushun,3 Toshi Oopun,Oono Shouji,Kitasanburakku,Kitamura Hiroshi,14.0,20.7,6.0
1329773,2015-09-21,Asahi Sentoraito Kinen,Asahi Sentoraito Kinen,3 Toshi Oopun,Oono Shouji,Kitasanburakku,Kitamura Hiroshi,1.0,12.5,6.0
1342304,2015-10-25,Kikkashou,Kikkashou,3 Toshi Oopun,Oono Shouji,Kitasanburakku,Kitamura Hiroshi,1.0,13.4,5.0
1331957,2015-12-27,Arimakinen,Arimakinen,3 Toshi Ijou Oopun,Oono Shouji,Kitasanburakku,Yokoyama Norihiro,3.0,8.4,4.0
1396145,2016-04-03,Sankeioosakahai,Sankeioosakahai,4 Toshi Ijou Oopun,Oono Shouji,Kitasanburakku,Taketoyo,2.0,6.2,5.0
1389990,2016-05-01,Tennoushou ( Haru ),Tennoushou ( Haru ),4 Toshi Ijou Oopun,Oono Shouji,Kitasanburakku,Taketoyo,1.0,4.5,2.0


In [4]:
# Simpan ke folder processed agar siap dipakai modeling
output_path = '../data/processed/race_result_cleaned.csv'
os.makedirs('../data/processed', exist_ok=True)

df_clean.to_csv(output_path, index=False)
print(f"Data bersih tersimpan di: {output_path}")

KeyboardInterrupt: 